# 量桨哈密顿量模块简介
*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

`Hamiltonian` 模块是量桨封装的与哈密顿量相关的模块。用户可以通过该模块，以泡利字符串的形式快速创建自定义哈密顿量，以及获取哈密顿量的各相关信息。

## 哈密顿量的创建

量子系统的哈密顿量是与系统总能量相关的可观测量，在 [变分量子本征值求解器](https://qml.baidu.com/tutorials/quantum-simulation/variational-quantum-eigensolver.html) 和 [利用Product Formula模拟时间演化](https://qml.baidu.com/tutorials/quantum-simulation/hamiltonian-simulation-with-product-formula.html) 等量子模拟算法中有着重要应用。在量子计算中，哈密顿量需要表示成量子计算机能处理的泡利算子张量积求和的形式。比如，如下哈密顿量就是以这种形式来表示的：

$$
H =  Z \otimes Z \otimes I +  I \otimes X \otimes Z
\tag{1}
$$

在 Paddle Quantum 中，用户可通过定义一个包含了多项泡利算子张量积信息的列表来创建哈密顿量，列表中的元素应为哈密顿量中每一项的系数及其表示了该项对应的泡利算符的字符串。如以上哈密顿量$H$中的第一项为$Z \otimes Z \otimes I$，对应的系数为$1$，对应的字符串为'Z0, Z1'，其中‘Z0’，‘Z1’分别表示该泡利算子作用在第$0$位量子比特和第$1$位量子比特上，这里的单位算符$I$可以被省略。接下来我们可以试着创建以上哈密顿量$H$。 


In [3]:
import numpy as np
import paddle_quantum
from paddle_quantum import Hamiltonian
from paddle_quantum.qinfo import random_hamiltonian_generator
from paddle_quantum.trotter import  get_1d_heisenberg_hamiltonian

h = Hamiltonian([[1,'Z0, Z1'], [1,'X1, Z2']])
print(h)

1.0 Z0, Z1
1.0 X1, Z2


## 哈密顿量支持的操作

Paddle Quantum 中的哈密顿量类 `Hamiltonian` 支持自动合并同类项，加减法，系数乘法，索引以及拆分等操作，如下：

In [5]:
h = Hamiltonian([[0.5,'Z0, Z1'], [0.5,'Z1, Z0'], [1,'X1, Z2']], compress = True)
print(h)

1.0 Z0, Z1
1.0 X1, Z2


In [6]:
h1 = Hamiltonian([[1,'Z0, Z1']])
h2 = Hamiltonian([[1 ,'X1, Z2']])
h = h1 + h2
print('h1 + h2: \n', h)
print('h1 * 2: ', h1 * 2)
print('h1: ', h[0])


h1 + h2: 
 1.0 Z0, Z1
1.0 X1, Z2
h1 * 2:  2.0 Z0, Z1
h1:  1.0 Z0, Z1


## 常用方法

`Hamiltonian` 内置的 `decompose_pauli_words()` 和 `decompose_with_sites()` 方法可以将哈密顿量中的各项泡利算符及对应的系数提取出来，分解为更加方便处理的形式。其中，`decompose_pauli_words()` 将哈密顿量分解为两部分，分别为系数列表和对应的泡利字符串 (Pauli strings) 列表。一个$n$比特的泡利算符的泡利字符串表示，是一个由n个来自于集合$\{'I', 'X', 'Y', 'Z'\}$的字符组成的字符串。如泡利算符$Z \otimes Z \otimes I$的泡利字符串表示为'ZZI'; `decompose_with_sites()`则将哈密顿量分解成三部分，分别是系数列表，泡利字符串的化简形式的列表以及它们分别作用的量子比特下标的列表。如泡利算符$I \otimes X \otimes Z$的泡利字符串表示为'IXZ'， 进一步的化简形式为'XZ', 由列表[1, 2]来分别标识'X'和'Z'分别作用的量子比特下标。

In [50]:
print('Pauli words 分解：', h.decompose_pauli_words())
print('Pauli with sites 分解：', h.decompose_with_sites())

Pauli words 分解： ([1.0, 1.0], ['ZZI', 'IXZ'])
Pauli with sites 分解： ([1.0, 1.0], ['ZZ', 'XZ'], [[0, 1], [1, 2]])


除此之外，`Hamiltonian` 内置的 `construct_h_matrix()` 方法 还可以创建其在泡利 $Z$ 基底下的矩阵形式：

In [51]:
h1.construct_h_matrix()

matrix([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]], dtype=complex64)

## 哈密顿量的其他创建方式

除了利用 `Hamiltonian`，用户还可以用量桨 `qinfo` 模块里的 `random_hamiltonian_generator` 函数来生成一个随机哈密顿量。如下，用户可以随机生成一个，作用在$3$个qubit上，总共有$4$项互不对易的泡利算符的哈密顿量：

In [57]:
h_random = random_hamiltonian_generator(num_qubits= 3, terms= 4)
print(h_random)

-0.588549459410759 Z0, Z2
-0.6863539010269364 X0, Z1, Y2
0.08533003337080869 Y0, X1, Y2
-0.002948614747939171 Z0, Z1, X2


用户还可以利用量桨 `trotter` 模块中的 `get_1d_heisenberg_hamiltonian` 函数来创建一个一维海森堡链的哈密顿量，实现 [一维海森堡链的自旋动力学模拟](https://qml.baidu.com/tutorials/quantum-simulation/simulate-the-spin-dynamics-on-a-heisenberg-chain.html):

In [62]:
h_heisenberg = get_1d_heisenberg_hamiltonian(length=5, j_x=1, j_y=2, j_z=2, h_z=2 * np.random.rand(5) - 1, periodic_boundary_condition=False)
print('系统的哈密顿量为：')
print(h_heisenberg)

系统的哈密顿量为：
1.0 X0, X1
2.0 Y0, Y1
2.0 Z0, Z1
1.0 X1, X2
2.0 Y1, Y2
2.0 Z1, Z2
1.0 X2, X3
2.0 Y2, Y3
2.0 Z2, Z3
1.0 X3, X4
2.0 Y3, Y4
2.0 Z3, Z4
-0.09133431514782564 Z0
-0.08305794685805723 Z1
0.23542270730722192 Z2
0.502365299254947 Z3
0.21580198180161436 Z4
